In [14]:
import requests

In [24]:
token='bde4a3deabdd81f9206e0a69974a1a8eb18425137adbadf8e9cccf9d45ba53bd' #
base_url='https://api.olhovivo.sptrans.com.br/v2.1'
response=requests.post(f'{base_url}/Login/Autenticar?token={token}')

In [25]:
headers={
    "Cookie": response.headers['Set-Cookie'] 
}

In [26]:
headers

{'Cookie': 'apiCredentials=728FF49050A120E52F3A5B9EC3C40CAF74189451AF3C694D47662747A69D5FE22EC3012D685012F7529B6B2B44F6B61D9A3935EABB3257082BE0A895C0380CF4AB7D1ED2E19997B4692652CCE3DBDF570C334E2AAB3DEFBEF033743ADABB5125B11089C7D0C6FE2B7F214F04418B02E592B9A05D8C6AEB8BC90DEC3DB9BACCEB236460DA314B1652B42C5CF92C3176E450B26D73969818B961A7FFD8016F28B8CA781879F808113381F59AAB0E47BFEAD688BF132A969552EF28D6C629E670D1ED2CF19E; path=/; HttpOnly; SameSite=Lax, __cf_bm=G4Pc.aAZAzpEcAadB_AsRKUqc_rF88_LC9ql19q7QMQ-1730720418-1.0.1.1-MgU.4BryXQU0d5yUnny4zlPYpLxyINlbKLrNX.dBz4fLEz8_nJQZ6hVb5a.dpP5D1CP9rHnfOklBsJ6A3HBoMw; path=/; expires=Mon, 04-Nov-24 12:10:18 GMT; domain=.sptrans.com.br; HttpOnly; Secure; SameSite=None'}

In [17]:
response=requests.get(f'{base_url}/Linha/Buscar?termosBusca=TERM', headers=headers)

In [21]:
response=requests.get(f'{base_url}/Parada/BuscarParadasPorLinha?codigoLinha=833', headers=headers) #get only linhas serving terminais

In [22]:
response.json()

[]

In [2]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 821.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 10.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 517.9 kB/s eta 0:00:000:01:00:01


In [6]:
import os
import boto3

s3_client=boto3.client(
    's3',
    endpoint_url=os.getenv("S3_ENDPOINT"),
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    config=boto3.session.Config(signature_version='s3v4')
)

In [62]:
import json
import boto3
from datetime import datetime

def save_to_s3(path_s3, data):
    bucket_name, key = path_s3.replace("s3a://", "").split('/', 1)
    current_timestamp=datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    print(f'saving {len(data)} items to {bucket_name} bucket in {key}dt_ingestion={current_timestamp}/data.json')
    s3_client.put_object(Body=(bytes(json.dumps(data, default=str).encode('UTF-8'))), Bucket=bucket_name, Key=f'{key}dt_ingestion={current_timestamp}/data.json')

    print('done!')

In [63]:
path_s3='s3a://bronze/linhas/'
save_to_s3(path_s3, response.json())

saving 1252 items to bronze bucket in linhas/dt_ingestion=2024-11-03_15-17-25/data.json
done!


In [13]:
linhas=s3_client.get_object(Bucket='silver', Key='/linhas/csv/*')
linhas_df=pandas.read_csv(linhas['Body'])

NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

In [12]:
response = s3_client.list_objects_v2(Bucket='silver')

# Check if the bucket has contents
if 'Contents' in response:
    for obj in response['Contents']:
        print(obj['Key'])  # Print the object's key (path/filename)
else:
    print("No objects found in the bucket.")

linhas/_delta_log/
linhas/data.csv/_SUCCESS
linhas/data.csv/part-00000-519e1e15-a6c4-4a61-ae37-b8bb3119ff8d-c000.csv
